In [7]:
import numpy as np
from scipy.spatial import ConvexHull
import matplotlib.pyplot as plt
import cv2
from scipy import ndimage
from skimage.measure import EllipseModel

ModuleNotFoundError: No module named 'cv2'

In [4]:
!pip install cv2
!pip install scipy.spatial
!pip install numpy
!pip install skiimage.measure

Defaulting to user installation because normal site-packages is not writeable


ERROR: Could not find a version that satisfies the requirement cv2 (from versions: none)
ERROR: No matching distribution found for cv2
You should consider upgrading via the 'c:\program files\python39\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable


ERROR: Could not find a version that satisfies the requirement scipy.spatial (from versions: none)
ERROR: No matching distribution found for scipy.spatial
You should consider upgrading via the 'c:\program files\python39\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'c:\program files\python39\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable


ERROR: Could not find a version that satisfies the requirement skiimage.measure (from versions: none)
ERROR: No matching distribution found for skiimage.measure
You should consider upgrading via the 'c:\program files\python39\python.exe -m pip install --upgrade pip' command.


from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def read_csv(csv_path):
    np_path_XYs = np.genfromtxt(csv_path, delimiter=',')
    path_XYs = []
    for i in np.unique(np_path_XYs[:, 0]):
        npXYs = np_path_XYs[np_path_XYs[:, 0] == i][:, 1:]
        XYs = []

        for j in np.unique(npXYs[:, 0]):
            XY = npXYs[npXYs[:, 0] == j][:, 1:]
            XYs.append(XY)

        path_XYs.append(XYs)

    return path_XYs

a=read_csv('/content/drive/MyDrive/adobe/gensolve/problems/isolated.csv')
print(len(a))
print(len(a[0]))
print(len(a[0][0]))
print(len(a[0][0][0]))

In [3]:
def plot(paths_XYs, colours):
    fig, ax = plt.subplots(tight_layout=True, figsize=(8, 8))

    for i, XYs in enumerate(paths_XYs):
        c = colours[i % len(colours)]

        for XY in XYs:
            ax.plot(XY[:, 0], XY[:, 1], c=c, linewidth=2)

    ax.set_aspect('equal')
    plt.show()
print(plot(a, ['r', 'g', 'b', 'y']))

NameError: name 'a' is not defined

In [ ]:
def plot_shape(paths_XYs, colours):
    fig, ax = plt.subplots(tight_layout=True, figsize=(8, 8))

    for i, XYs in enumerate(paths_XYs):
        c = colours[i % len(colours)]

        ax.plot(XYs[:, 0], XYs[:, 1], c=c, linewidth=2)

    ax.set_aspect('equal')
    plt.show()
print(plot_shape(a[0], ['r', 'g', 'b', 'y']))
print(plot_shape(a[1], ['r', 'g', 'b', 'y']))
print(plot_shape(a[2], ['r', 'g', 'b', 'y']))

In [ ]:
def identify_shape(points):
    print(len(points))
    if len(points) < 3:
        return 'unknown'

    hull = ConvexHull(points)
    hull_area = hull.volume
    hull_perimeter = calculate_perimeter(hull)

    if hull_perimeter == 0:
        return 'unknown'

    compactness = 4 * np.pi * hull_area / (hull_perimeter ** 2)

    print(f"Compactness: {compactness}, Hull Area: {hull_area}, Hull Perimeter: {hull_perimeter}")

    if compactness > 0.88:
        print("Identified as circle")
        return 'circle'

    aspect_ratio = calculate_aspect_ratio(points)
    if 0.8 < aspect_ratio < 1.2:
        print("Identified as rectangle")
        return 'rectangle'

    if compactness < 0.7:
        print("Identified as star")
        return 'star'

    if 0.7 < compactness <= 0.88:
        print("Identified as ellipse")
        return 'ellipse'

    if compactness < 1e-3:
        print("Identified as line")
        return 'line'

    num_vertices = len(hull.vertices)
    if 4 <= num_vertices <= 10:
        aspect_ratio = calculate_aspect_ratio(points)
        if aspect_ratio < 1.2:
            shapes = {4: 'square', 5: 'pentagon', 6: 'hexagon', 7: 'heptagon', 8: 'octagon', 9: 'nonagon', 10: 'decagon'}
            shape_name = shapes.get(num_vertices, 'polygon')
            print(f"Identified as {shape_name}")
            return shape_name

    if len(hull.vertices) > 10:
        print("Identified as polygon")
        return 'polygon'

    return 'unknown'


In [ ]:
def regularize_line(points):
    if len(points) < 2:
        return np.array([])
    p1, p2 = points[0], points[-1]
    x = np.array([p1[0], p2[0]])
    y = np.array([p1[1], p2[1]])

    return np.column_stack((x, y))

In [ ]:

def regularize_polygon(points):
    center = np.mean(points, axis=0)
    radius = np.mean(np.linalg.norm(points - center, axis=1))
    num_sides = len(points)

    angles = np.linspace(0, 2 * np.pi, num_sides, endpoint=False)
    x = center[0] + radius * np.cos(angles)
    y = center[1] + radius * np.sin(angles)

    return np.column_stack((x, y))

In [ ]:
def regularize_circle(points, max_width, max_height, scale_factor=0.9):
    center = np.mean(points, axis=0)
    radius = np.median(np.linalg.norm(points - center, axis=1))

    min_dimension = min(max_width, max_height)
    scale = (min_dimension * scale_factor) / (2 * radius)

    num_points = len(points)
    theta = np.linspace(0, 2 * np.pi, num_points, endpoint=False)
    x = center[0] + radius * scale * np.cos(theta)
    y = center[1] + radius * scale * np.sin(theta)

    return np.column_stack((x, y))

In [ ]:
def regularize_ellipse(points):
    if len(points) < 5:
        return np.array([])

    ellipse = EllipseModel()
    ellipse.estimate(points)
    xc, yc, a, b, theta = ellipse.params

    num_points = len(points)
    theta_values = np.linspace(0, 2 * np.pi, num_points)
    x = xc + a * np.cos(theta_values) * np.cos(theta) - b * np.sin(theta_values) * np.sin(theta)
    y = yc + a * np.cos(theta_values) * np.sin(theta) + b * np.sin(theta_values) * np.cos(theta)

    return np.column_stack((x, y))

In [ ]:
def regularize_rectangle(points, max_width, max_height, scale_factor=0.9):
    min_coords = np.min(points, axis=0)
    max_coords = np.max(points, axis=0)

    width = max_coords[0] - min_coords[0]
    height = max_coords[1] - min_coords[1]

    scale_x = (max_width * scale_factor) / width
    scale_y = (max_height * scale_factor) / height

    center = np.mean(points, axis=0)

    x = [min_coords[0], max_coords[0], max_coords[0], min_coords[0], min_coords[0]]
    y = [min_coords[1], min_coords[1], max_coords[1], max_coords[1], min_coords[1]]

    x = center[0] + (np.array(x) - center[0]) * scale_x
    y = center[1] + (np.array(y) - center[1]) * scale_y

    return np.column_stack((x, y))


In [ ]:
def regularize_star(points):
    center = np.mean(points, axis=0)
    max_radius = np.max(np.linalg.norm(points - center, axis=1))
    min_radius = max_radius * 0.4

    num_points = 10
    angles = np.linspace(0, 2 * np.pi, num_points, endpoint=False)
    radii = np.zeros(num_points)
    radii[::2] = max_radius
    radii[1::2] = min_radius

    x = center[0] + radii * np.cos(angles)
    y = center[1] + radii * np.sin(angles)

    x = np.append(x, x[0])
    y = np.append(y, y[0])

    return np.column_stack((x, y))

In [ ]:
def calculate_max_extents(points):
    min_coords = np.min(points, axis=0)
    max_coords = np.max(points, axis=0)

    extents = max_coords - min_coords

    return extents[0], extents[1]

In [ ]:
def calculate_aspect_ratio(points):
    hull = ConvexHull(points)
    hull_points = hull.points[hull.vertices]


    dists = [np.linalg.norm(hull_points[i] - hull_points[(i + 1) % len(hull_points)]) for i in range(len(hull_points))]
    width, height = sorted(dists)[:2]

    return max(width, height) / min(width, height)

In [ ]:
def regularize_shape(points):
    if len(points) == 0:
        return np.array([])

    max_width, max_height = calculate_max_extents(points)
    shape_type = identify_shape(points)

    if shape_type == 'circle':
        return regularize_circle(points, max_width, max_height)
    elif shape_type == 'rectangle':
        return regularize_rectangle(points, max_width, max_height)
    elif shape_type == 'ellipse':
        return regularize_ellipse(points)
    elif shape_type == 'line':
        return regularize_line(points)
    elif shape_type == 'square':
        return regularize_polygon(points)
    elif shape_type == 'pentagon':
        return regularize_polygon(points)
    elif shape_type == 'hexagon':
        return regularize_polygon(points)
    elif shape_type == 'heptagon':
        return regularize_polygon(points)
    elif shape_type == 'octagon':
        return regularize_polygon(points)
    elif shape_type == 'nonagon':
        return regularize_polygon(points)
    elif shape_type == 'decagon':
        return regularize_polygon(points)
    elif shape_type == 'star':
        return regularize_star(points)
    elif shape_type == 'polygon':
        return regularize_polygon(points)
    else:
        return regularize_polygon(points)

In [ ]:
colours = ['r', 'g', 'b', 'y']
pathXY=read_csv('/content/drive/MyDrive/adobe/gensolve/problems/isolated.csv')
regularized_path_XYs = []
plot(pathXY,colours)

for path in pathXY:

    regularized_path = []
    for shape in path:

        regularized_shape = regularize_shape(shape)
        regularized_path.append(regularized_shape)

    regularized_path_XYs.append(regularized_path)
plot(regularized_path_XYs, colours)